In [1]:
import os

In [2]:
def get_recursive_filenames(directory,upc_to_filenames):
    for name in os.listdir(directory):
        path = os.path.join(directory, name)
        if os.path.isdir(path):
            get_recursive_filenames(path,upc_to_filenames)
        else:
            upc = os.path.basename(os.path.dirname(path))
            if upc in upc_to_filenames:
                upc_to_filenames[upc].append(path)
            else:
                upc_to_filenames[upc] = [path]


In [3]:
upc_to_filenames={}
get_recursive_filenames('/home/src/goodsdl/media/dataset/step20',upc_to_filenames)

In [5]:
len(upc_to_filenames)

423

In [1]:
# source_cluster = [['A','B'],['A','D'],['B','E'],['E','F'],['C','D'],['G','A'],['N','M']]
source_cluster = [['6951003200048', '6923976111171'], ['6923976111195', '6923976113137'], ['6924743920361', '6921290410444'], ['6924743919228', '6924743920996'], ['6924743919228', '6924743919242'], ['6924743920996', '6924743919242'], ['6909409040775', '6909409012321'], ['6909409040898', '6923976111171'], ['6920912342019', '6920912342002'], ['6932005203077', '6943290500666'], ['6924743919235', '6923976111171'], ['6924743919235', '6924743919242'], ['6924743918610', '6923976111171']]
upc_to_lines = {}  # {A:[0,1,5],B:[0,2],C:[3],D:[1,3],E:[2,4],F:[4],G:[5],N:[6],M:[6]}
for i in range(len(source_cluster)):
    for j in range(2):
        if source_cluster[i][j] in upc_to_lines:
            upc_to_lines[source_cluster[i][j]].append(i)
        else:
            upc_to_lines[source_cluster[i][j]] = [i]

print('upc_to_lines:')
print(upc_to_lines)


upc_to_lines:
{'6951003200048': [0], '6923976111171': [0, 7, 10, 12], '6923976111195': [1], '6923976113137': [1], '6924743920361': [2], '6921290410444': [2], '6924743919228': [3, 4], '6924743920996': [3, 5], '6924743919242': [4, 5, 11], '6909409040775': [6], '6909409012321': [6], '6909409040898': [7], '6920912342019': [8], '6920912342002': [8], '6932005203077': [9], '6943290500666': [9], '6924743919235': [10, 11], '6924743918610': [12]}


In [3]:
solved_line_to_upc = {}
solved_upc_to_lines = {}  # {A:[0,1,2,3,4,5],N:[6]}
for upc in upc_to_lines:
    unsolved_lines = []
    cluster_upcs = []
    for line in upc_to_lines[upc]:
        if line in solved_line_to_upc:
            cluster_upc = solved_line_to_upc[line]
            if cluster_upc not in cluster_upcs:
                cluster_upcs.append(cluster_upc)
            if line not in solved_upc_to_lines[cluster_upc]:
                solved_upc_to_lines[cluster_upc].append(line)
        else:
            unsolved_lines.append(line)

    if len(cluster_upcs) > 1: #需要处理多个聚类
        cluster_upcs = sorted(cluster_upcs)
        main_upc = cluster_upc[0]
        if main_upc not in solved_upc_to_lines:
            solved_upc_to_lines[main_upc] = []
        for cluster_upc in cluster_upcs:
            if cluster_upc != main_upc:
                for line in solved_upc_to_lines[cluster_upc]:
                    if line not in solved_upc_to_lines[main_upc]:
                        solved_upc_to_lines[main_upc].append(line)
                    solved_line_to_upc[line] = main_upc
                solved_upc_to_lines.pop(cluster_upc)            
        cluster_upcs = [main_upc]
    for line in unsolved_lines:
        if len(cluster_upcs) == 0: # 不聚类
            solved_line_to_upc[line] = upc
            if upc not in solved_upc_to_lines:
                solved_upc_to_lines[upc] = [line]
            else:
                if line not in solved_upc_to_lines[upc]:
                    solved_upc_to_lines[upc].append(line)
        elif len(cluster_upcs) == 1: # 聚到指定的类
            if line not in solved_upc_to_lines[cluster_upcs[0]]:
                solved_upc_to_lines[cluster_upcs[0]].append(line)
            solved_line_to_upc[line] = cluster_upcs[0]

print('solved_upc_to_lines:')
print(solved_upc_to_lines)
print('solved_line_to_upc:')
print(solved_line_to_upc)



solved_upc_to_lines:
{'6923976111195': [1], '6924743920361': [2], '6909409040775': [6], '6920912342019': [8], '6932005203077': [9], '6': [3, 4, 5, 11, 0, 7, 10, 12]}
solved_line_to_upc:
{0: '6', 7: '6', 10: '6', 12: '6', 1: '6923976111195', 2: '6924743920361', 3: '6', 4: '6', 5: '6', 11: '6', 6: '6909409040775', 8: '6920912342019', 9: '6932005203077'}


In [4]:
solved_cluster = {}  # {'A': ['B', 'C', 'D', 'G', 'H', 'E', 'F'], 'M': ['N']}

for key in solved_upc_to_lines:
    one_cluster = []
    for line in solved_upc_to_lines[key]:
        for upc in source_cluster[line]:
            if upc not in one_cluster:
                one_cluster.append(upc)

    one_cluster = sorted(one_cluster)
    main_upc = one_cluster[0]
    one_cluster.remove(main_upc)
    solved_cluster[main_upc] = one_cluster

print('solved_cluster:')
print(solved_cluster)


solved_cluster:
{'6923976111195': ['6923976113137'], '6921290410444': ['6924743920361'], '6909409012321': ['6909409040775'], '6920912342002': ['6920912342019'], '6932005203077': ['6943290500666'], '6909409040898': ['6923976111171', '6924743918610', '6924743919228', '6924743919235', '6924743919242', '6924743920996', '6951003200048']}
